In [7]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

indisponivel = "indisponivel"

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [9]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [10]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [11]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt
0,30.14.117,"APITO, MATERIAL PLÁSTICO, APLICAÇÃO ESPORTE, T...",16,174.84,10.93
1,30.14.120,BOLA TENIS DE MESA,17,33.83,1.99
2,30.14.121,REDE ESPORTE BASQUETE,8,127.92,15.99
3,30.14.122,REDE ESPORTE FUTSAL,10,1099.40,109.94
4,30.14.123,REDE ESPORTE FUTEBOL DE CAMPO,3,261.00,87.00
...,...,...,...,...,...
625,30.42.972,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,39,208.65,5.35
626,30.42.973,LUZ EMERGÊNCIA BOTOEIRA INDUSTRIAL DE COMANDO ...,40,214.00,5.35
627,30.44.67,"Fita adesiva, material propileno, 20mm x 30m, ...",12,267.60,22.30
628,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,2,43.00,21.50
